# Datenaufbereitung

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [62]:
df = pd.read_parquet("2023-03-25_gesamt_ocr_rhein_neckar.parquet")
df = df.rename(columns={'doc_typ': 'doc_typ_old'})

## Dokumenten-Typ berabeiten

In [ ]:
doc_typ = df['doc_typ'].unique()
df_doc_typ = pd.DataFrame(doc_typ)
df_doc_typ.to_csv('doc_typ.csv')

In [ ]:
doc_typ_2 = df[(pd.isnull(df['doc_typ_old']) == True) | (df['doc_typ_old'] == "Sitzungsunterlagen und Beschlüsse")]
doc_typ_2.to_csv('doc_typ_2.csv')

In [64]:
doc_typ_reclass = pd.read_excel("doc_typ_reclass.xlsx")
doc_typ_2_reclass = pd.read_excel("doc_typ_2_reclass.xlsx")

In [65]:
df = df.merge(doc_typ_reclass, on='doc_typ_old', how="left")
df = df.merge(doc_typ_2_reclass, left_index=True, right_on='index', how='left')
df['doc_typ'] = df['doc_typ_x'].combine_first(df['doc_typ_y'])
df.drop(['doc_typ_old','doc_typ_x','doc_typ_y', 'doc_name_y' ,'id', 'index'], axis=1, inplace=True)

In [72]:
df['doc_typ'] = df['doc_typ'].astype('category')
df.to_parquet("2023-04-03_gesamt_rhein_neckar.parquet")

In [12]:
df['doc_typ'].value_counts()

Vorlage             26793
Anhang Vorlage      25528
Beschluss            4738
Niederschrift        4690
Anhang Sitzung       1644
Bekanntmachung       1598
Anfrage              1117
Antrag                881
Anhang Beschluss      777
Name: doc_typ, dtype: int64

## Mobilitätsbezug (ja/nein)

In [76]:
bag_of_words = ["verkehr", "mobilität", 
                "miv","automobil", "pkw", "personenkraftw", "kfz", "kraftfahrzeug", "lkw", "motorrad",
                "stvo", "höchstgeschwindigkeit","führerschein",
                "modal split", "umweltverbund",
                "tankstelle", "ladesäule",
                "ampel", "lsa", "lichtsignalanlage",
                # "stau", "cargo", "logistik",
                "fahrrad", "radverkehr", "schutzstreifen", "radfahr", "abstellanlage", "lastenrad",
                "zu fuß", "fußgäng", "gehweg", "zufußgehen",
                "öpnv", "bus", "bahn", "tram", "haltestelle",
                "car", "bike" ,"roller", "scooter", "sharing",
                "parken", "parkpl", "stellpl", "parkhaus", "garage",
                ]

In [77]:
df['verkehrsbezug'] = df['content'].str.contains('|'.join(bag_of_words), case=False)

In [16]:
df['verkehrsbezug'].value_counts()

False    41090
True     28041
Name: verkehrsbezug, dtype: Int64

In [80]:
df.to_parquet("2023-04-03_gesamt_rhein_neckar.parquet")

## Beschlussstatus

In [86]:
import re

In [14]:
pd.notnull(df['status']).value_counts()

False    52812
True     16605
Name: status, dtype: int64

In [83]:
status = df['status'].unique()
df_status = pd.DataFrame(status)
df_status.to_excel('status.xlsx')

In [96]:
status_list = [r"Beschluss:\s*(\d+\s[-(),\w]+\s*)*",
               "abgelehnt",
               "abgesetzt",
               r"mehrheitlich(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}"
               "Absetzung von der Tagesordnung",
               "einstimmig(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}",
               r"mehrheitlich(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}",
               r"keine Einigung(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}",
               "ohne Abstimmung",
               "nicht in der Tagesordnung aufgenommen",
               "teilweise",
               "Vertagung",
               "vertagt"
               "zur Kenntnis genommen",
               r"zurückgestellt(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}",
               "zurückgewiesen",
               r"Zurückweisung(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}",
               "offen"

               ]

status_re = r"(Beschluss(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,50})"

In [112]:
no_status = df[pd.isnull(df['status']) == True]

In [120]:
df = df.reset_index()

In [122]:
no_status_filtered = no_status[no_status['doc_typ'] == ('Beschluss' or'Vorlage' or'Antrag'or'Anfrage') ]

In [129]:
df.drop('status_extract', axis=1, inplace=True)

In [132]:
status_extract_all = no_status_filtered['content'].str.extractall(status_re, flags=re.IGNORECASE)

In [126]:
df_status_extract = df[pd.notnull(df['status_extract']) == True]

In [127]:
status_extract_all.to_csv("status_extract_all.csv")

In [128]:
df['content'][23106]

'S T A D T  L A M P E R T H E I M  \nLampertheim, den 24.09.2007 \n \n \n \nB E S C H L U S S  N r .   \n \nder Sitzung der Stadtverordnetenversammlung der Stadt Lampertheim \n \nvom Freitag, den 14.09.2007 um 19:03 Uhr \n \nim Sitzungssaal des Stadthauses, Römerstraße 102, 68623 Lampertheim \n \nEs waren anwesend: \nStass, Brigitte - Stadtverordnetenvorsteherin  \nBauer, Lothar - Stadtverordneter \n \nBerg, Karl-Heinz - Stadtverordneter  \nBittner, Thomas - Stadtverordneter  \nBlepp, Dieter - Stadtverordneter \n \nBrechenser, Dieter - Stadtverordneter \n \nBuschmann, Irma - Stadtverordnete  \nEbert, Rita - Stadtverordnete  \nGalvagno, Nunzio - Stadtverordneter \n \nGötz, Fritz - Stadtverordneter \n \nHahn, Hans - Stadtverordneter \n \nHartmann, Sabine - Stadtverordnete  \nHofmann, Werner - Stadtverordneter \n \nHorstfeld, Karl-Heinz - Stellvertretender Stadtverordnetenvorsteher \n \nHummel, Helmut - Stadtverordneter  \nJacobi, Michael - Stadtverordneter  \nKirsch, Walter - Stadtverord

## Allgemeine Informationen über Datensatz

In [2]:
df = pd.read_parquet("2023-04-03_gesamt_rhein_neckar.parquet")

In [3]:
df['kommune'] = df["kommune"].cat.rename_categories({"Kreis_Heilbronn": "Kreis Heilbronn",
                          "Rhein_Neckar_Kreis": "Rhein-Neckar-Kreis",
                          "Bad Duerkheim": "Bad Dürkheim",
                          })

In [4]:
df = df[df["date"] < "2022-12-31"]

In [5]:
df['date_print'] = df['date'].dt.strftime('%d.%m.%Y')

In [43]:
df_date = pd.crosstab(index=df['kommune'], columns='Aggregat', values=df['date'], aggfunc=[min,max])

In [44]:
df_date['diff_years'] = (df_date['max'] - df_date['min']) / np.timedelta64(1,'Y')

In [47]:
df_date

,min,max,diff_years
col_0,Aggregat,Aggregat,
kommune,,,
Bad Dürkheim,2006-12-12,2022-12-13,16.003066
Bad Friedrichshall,2011-02-22,2022-12-16,11.814069
Bad Rappenau,2016-01-28,2022-12-15,6.880360
Bensheim,2014-02-01,2022-07-01,8.410850
Frankenthal,2004-07-07,2022-12-14,18.437066
Hassloch,2010-04-28,2022-12-14,12.629965
Heppenheim,2012-05-15,2022-12-01,10.546418
Hockenheim,2018-01-22,2022-12-14,4.892640


In [48]:
df_date.to_latex("../../latex/Tables/Kommunen_Datum_Min_Max_Duration.tex", 
                caption="Zeitspanne der erhobenen Daten",
                label="tab:zeitspanne_daten",
                column_format="lccr"
                )

C:\Users\johan\AppData\Local\Temp\ipykernel_14556\3816574112.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_date.to_latex("../../latex/Tables/Kommunen_Datum_Min_Max_Duration.tex",


In [11]:
df['doc_typ'].value_counts()

Vorlage             26793
Anhang Vorlage      25528
Beschluss            4738
Niederschrift        4690
Anhang Sitzung       1644
Bekanntmachung       1598
Anfrage              1117
Antrag                881
Anhang Beschluss      777
Name: doc_typ, dtype: int64

In [ ]:
df['doc_typ'].value_counts().to_latex("../../latex/Tables/dokumententypen.tex", 
                caption="Dokumententypen des gesamten Datensatzes",
                label="tab:dokumententypen",
                column_format="lr"
                )

In [15]:
df['verkehrsbezug'] = df['verkehrsbezug'].astype('boolean')

In [50]:
df.to_parquet('2023-05-22_gesamtdaten_bis_2022.parquet')

In [6]:
df = pd.read_parquet('2023-05-22_gesamtdaten_bis_2022.parquet')

In [18]:
pd.isnull(df['content']).value_counts()

False    69131
True       286
Name: content, dtype: int64

In [19]:
df.head()

,content,date,doc_name_x,file_urls,gremium,kommunale_ebene,kommune,pdf_name,rel_path_to_file,sitzung_nr,status,top_name,top_nr,vorlage_nr,ocr,doc_typ,verkehrsbezug,date_print
index,,,,,,,,,,,,,,,,,,
15814.0,\n \n \nBeschlußvorlage \nöffentlich \n \nFac...,2007-02-13,Vorlage,https://sessionnet.krz.de/bad-duerkheim/bi/get...,Stadtrat,Stadt,Bad Dürkheim,2007-02-13_Bad_Duerkheim_60212.pdf,Dateien/Kommunen/Bad_Duerkheim/2007-02-13_Bad_...,60212.0,None,Tätigkeitsbericht der Agenda 21,2.0,20070019/1.1,False,Vorlage,True,13.02.2007
15815.0,\n \n \nBeschlußvorlage \nnicht öffentlich \n...,2007-02-13,Vorlage,https://sessionnet.krz.de/bad-duerkheim/bi/get...,Stadtrat,Stadt,Bad Dürkheim,2007-02-13_Bad_Duerkheim_60204.pdf,Dateien/Kommunen/Bad_Duerkheim/2007-02-13_Bad_...,60204.0,None,Prüfung der Stadtkasse hier: Unterrichtung übe...,3.0,20070018/1.7,False,Vorlage,False,13.02.2007
21130.0,Unvermutete überörtliche Prüfung der Stadtkass...,2007-02-13,Auszug aus Bericht Kassenprüfung,https://sessionnet.krz.de/bad-duerkheim/bi/get...,Stadtrat,Stadt,Bad Dürkheim,2007-02-13_Bad_Duerkheim_60220.pdf,Dateien/Kommunen/Bad_Duerkheim/2007-02-13_Bad_...,60220.0,None,Prüfung der Stadtkasse hier: Unterrichtung übe...,3.0,20070018/1.7,True,Anhang Vorlage,True,13.02.2007
15816.0,\n \n \nBeschlußvorlage \nnichtöffentlich \n ...,2007-02-13,Vorlage,https://sessionnet.krz.de/bad-duerkheim/bi/get...,Stadtrat,Stadt,Bad Dürkheim,2007-02-13_Bad_Duerkheim_60195.pdf,Dateien/Kommunen/Bad_Duerkheim/2007-02-13_Bad_...,60195.0,None,Neubesetzung städtischer Ausschüsse,5.0,20070014/FB1,False,Vorlage,False,13.02.2007
28204.0,"""REPUBLIKANER R EP\n\nStadtratsfraktion Bad Dü...",2007-02-13,Schreiben der REP-Stadtratsfraktion vom 10.01....,https://sessionnet.krz.de/bad-duerkheim/bi/get...,Stadtrat,Stadt,Bad Dürkheim,2007-02-13_Bad_Duerkheim_60213.pdf,Dateien/Kommunen/Bad_Duerkheim/2007-02-13_Bad_...,60213.0,None,Neubesetzung städtischer Ausschüsse,5.0,20070014/FB1,True,Anhang Vorlage,False,13.02.2007


In [7]:
# Wenn es keinen Inhalt gibt, dann auslassen
df = df[df['content'].notna()]

In [21]:
pd.isnull(df['content']).value_counts()

False    69131
Name: content, dtype: int64

In [6]:
pd.crosstab(index=df['kommune'], columns=df['doc_typ']).to_latex("../../latex/Tables/Kommunen_Dokumententypen.tex", 
                caption="Dokumententyp je Kommune",
                label="tab:kommune dokumententypen",
                )

C:\Users\johan\AppData\Local\Temp\ipykernel_14556\3476177009.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  pd.crosstab(index=df['kommune'], columns=df['doc_typ']).to_latex("../../latex/Tables/Kommunen_Dokumententypen.tex",


In [12]:
pd.crosstab(index=df['kommune'], columns='Count')

col_0,Count
kommune,
Bad Dürkheim,1817
Bad Friedrichshall,2401
Bad Rappenau,1402
Bensheim,56
Frankenthal,7368
Hassloch,2716
Heppenheim,2660
Hockenheim,1746
Homburg,1006


In [13]:
pd.crosstab(index=df['kommune'], columns='Count').to_latex("../../latex/Tables/Kommunen_Anzahl_Dokumente.tex", 
                caption="Anzahl der Dokumente pro Kommune",
                label="tab:kommunen_anzahl_dokumente",
                )

C:\Users\johan\AppData\Local\Temp\ipykernel_14556\1050045831.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  pd.crosstab(index=df['kommune'], columns='Count').to_latex("../../latex/Tables/Kommunen_Anzahl_Dokumente.tex",


In [10]:
pd.crosstab(index=df['kommune'], columns="Aggregat", values=df['verkehrsbezug'], aggfunc=[sum])

,sum
col_0,Aggregat
kommune,
Bad Dürkheim,897
Bad Friedrichshall,768
Bad Rappenau,568
Bensheim,56
Frankenthal,2799
Hassloch,1068
Heppenheim,1337
Hockenheim,846


In [11]:
pd.crosstab(index=df['kommune'], columns="Aggregat", values=df['verkehrsbezug'], aggfunc=[sum]).to_latex("../../latex/Tables/Kommunen_Dokumente_Verkehrsbezug.tex", 
                caption="Anzahl der Dokumente mit Verkehrsbezug pro Kommune",
                label="tab:kommunen_anzahl_dokumente_verkehr",
                )

C:\Users\johan\AppData\Local\Temp\ipykernel_14556\1696818246.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  pd.crosstab(index=df['kommune'], columns="Aggregat", values=df['verkehrsbezug'], aggfunc=[sum]).to_latex("../../latex/Tables/Kommunen_Dokumente_Verkehrsbezug.tex",


In [8]:
frankenthal = df[df['kommune'] == 'Frankenthal']

In [ ]:
frankenthal.to_parquet('frankthal.parquet')